In [62]:
import pandas as pd

In [63]:
data = pd.read_csv('./fraudTrain.csv')

In [64]:
# import numpy as np
# from keras.models import Sequential
# from keras.layers import Dense, LeakyReLU
# from keras.optimizers import Adam
# from sklearn.utils import shuffle

# def build_generator(latent_dim):
#     model = Sequential()
#     model.add(Dense(32, input_dim=latent_dim))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dense(64))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dense(128))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dense(22, activation='tanh'))
#     return model

# def build_discriminator():
#     model = Sequential()
#     model.add(Dense(64, input_dim=30))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dense(32))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dense(1, activation='sigmoid'))
#     return model

# def build_gan(generator, discriminator):
#     discriminator.trainable = False
#     model = Sequential()
#     model.add(generator)
#     model.add(discriminator)
#     model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
#     return model

# data = pd.read_csv('./fraudTrain.csv')
# X_train, y_train = data.iloc[:, :-1].values, data.iloc[:, -1].values

# batch_size = 128
# epochs = 1000

# latent_dim = 10

# generator = build_generator(latent_dim)
# discriminator = build_discriminator()
# gan = build_gan(generator, discriminator)

# for epoch in range(epochs):
#     noise = np.random.normal(0, 1, (batch_size, latent_dim))
#     gen_samples = generator.predict(noise)
#     X = np.concatenate((X_train, gen_samples))
#     y = np.concatenate((y_train, np.zeros((batch_size, 1))))
#     X, y = shuffle(X, y)
#     discriminator.trainable = True
#     discriminator.train_on_batch(X, y)
#     noise = np.random.normal(0, 1, (batch_size, latent_dim))
#     y_gen = np.ones((batch_size, 1))
#     discriminator.trainable = False
#     gan.train_on_batch(noise, y_gen)

# noise = np.random.normal(0, 1, (batch_size, latent_dim))
# gen_samples = generator.predict(noise)

# X_augmented = np.concatenate((X_train, gen_samples))
# y_augmented = np.concatenate((y_train, np.ones((batch_size, 1))))

# X_augmented, y_augmented = shuffle(X_augmented, y_augmented)

In [65]:
data = data.drop(['Unnamed: 0', 'city_pop', 'job', 'street', 'gender', 'first', 'last', 'dob', 'unix_time', 'city', 'state', 'zip', 'category'], axis=1)

In [66]:
data['trans_date_trans_time'] = data['trans_date_trans_time'].apply(lambda x: pd.to_datetime(x))

In [67]:
data['last_transaction_time'] = data.groupby('cc_num')['trans_date_trans_time'].transform('max')


In [68]:
data['last_customer_lat'] = data.groupby('cc_num')['lat'].transform('last')
data['last_customer_long'] = data.groupby('cc_num')['long'].transform('last')

data['last_merchant_lat'] = data.groupby('merchant')['lat'].transform('last')
data['last_merchant_long'] = data.groupby('merchant')['long'].transform('last')

In [71]:
import math

def calculate_distance(lat_prev, long_prev, lat_current, long_current):
    R = 6371  
    lat_prev_rad = math.radians(lat_prev)
    long_prev_rad = math.radians(long_prev)
    lat_current_rad = math.radians(lat_current)
    long_current_rad = math.radians(long_current)

    delta_lat = lat_current_rad - lat_prev_rad
    delta_long = long_current_rad - long_prev_rad

    a = math.sin(delta_lat / 2) ** 2 + math.cos(lat_prev_rad) * math.cos(lat_current_rad) * math.sin(delta_long / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

In [73]:
data['customer_distance'] = data.apply(lambda x: calculate_distance(x['last_customer_lat'], x['last_customer_long'], x['lat'], x['long']), axis=1)
data['merchant_distance'] = data.apply(lambda x: calculate_distance(x['last_merchant_lat'], x['last_merchant_long'], x['merch_lat'], x['merch_long']), axis=1)

In [77]:
data['merch_score'] = data.groupby('merchant')['is_fraud'].transform('mean')
data['customer_score'] = data.groupby('cc_num')['is_fraud'].transform('mean')

In [81]:
data = data.drop(['cc_num', 'merchant', 'trans_num', 'trans_date_trans_time'], axis = 1)

In [84]:
data

,amt,lat,long,merch_lat,merch_long,is_fraud,last_transaction_time,last_customer_lat,last_customer_long,last_merchant_lat,last_merchant_long,customer_distance,merchant_distance,merch_score,customer_score
0,4.97,36.0788,-81.1781,36.011293,-82.048315,0,2020-06-21 06:08:15,36.0788,-81.1781,30.4066,-91.1468,0.0,1050.310091,0.014207,0.000000
1,107.23,48.8878,-118.2105,49.159047,-118.186462,0,2020-06-21 07:59:13,48.8878,-118.2105,40.6025,-80.3863,0.0,3089.902532,0.010787,0.000000
2,220.11,42.1808,-112.2620,43.150704,-112.154481,0,2020-06-20 07:19:00,42.1808,-112.2620,43.9446,-88.0911,0.0,1934.506583,0.002111,0.000000
3,45.00,46.2306,-112.1138,47.034331,-112.561071,0,2020-06-19 01:48:42,46.2306,-112.1138,39.6967,-78.8031,0.0,2823.538028,0.003444,0.030426
4,41.96,38.4207,-79.4629,38.674999,-78.632459,0,2020-06-21 03:35:31,38.4207,-79.4629,34.3470,-87.7154,0.0,942.996194,0.003769,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,15.56,37.7175,-112.4777,36.841266,-111.690765,0,2020-06-21 12:12:08,37.7175,-112.4777,37.7175,-112.4777,0.0,119.752136,0.002622,0.005948
1296671,51.70,39.2667,-77.5101,38.906881,-78.246528,0,2020-06-21 12:12:19,39.2667,-77.5101,39.2667,-77.5101,0.0,75.104085,0.002284,0.015066
1296672,105.93,32.9396,-105.8189,33.619513,-105.130529,0,2020-06-21 12:12:32,32.9396,-105.8189,32.9396,-105.8189,0.0,99.047734,0.001079,0.005314
1296673,74.90,43.3526,-102.5411,42.788940,-103.241160,0,2020-06-21 12:13:36,43.3526,-102.5411,43.3526,-102.5411,0.0,84.627652,0.002094,0.003953


In [85]:
data.to_csv('preprocessed_fraudTrain.csv')